In [23]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [24]:
#define current city
city = 'newberg'

In [25]:
#read in labels
labels = pd.read_csv('../../../data/{}/processed-labels/labels_raw_{}.csv'.format(city, city, city))
labels

,label_id,gsv_panorama_id_x,label_type,severity,correct,has_validations,expired,high_quality_user,geometry,gsv_panorama_id_y,pano_width,pano_height,pano_x,pano_y,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng
0,145,0R2JTLibSCNsNb7aGNUzWg,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.9529037475586 45.317649841308594),0R2JTLibSCNsNb7aGNUzWg,13312.0,6656.0,9123,3863,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904
1,18134,Z6Ja9-4wJxpQZPEnn8KSXA,NoSidewalk,4.0,NaN,False,False,True,POINT (-122.96400451660156 45.313819885253906),Z6Ja9-4wJxpQZPEnn8KSXA,13312.0,6656.0,6644,3744,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005
2,151,oqg9ULHZul23xfU6DwTRNg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.95359802246094 45.3176155090332),oqg9ULHZul23xfU6DwTRNg,13312.0,6656.0,7788,3776,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598
3,153,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.95402526855469 45.317420959472656),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,4663,3709,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025
4,154,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.95418548583984 45.31760025024414),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,7268,3600,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,5KKnsHQgODUBbIRZYiVbqg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.92579650878906 45.30332946777344),5KKnsHQgODUBbIRZYiVbqg,13312.0,6656.0,256,3757,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797
16843,8272,mdbgEoi-XTfZGiKqcWMlqw,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.92559051513672 45.29985809326172),mdbgEoi-XTfZGiKqcWMlqw,3328.0,1664.0,2697,986,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591
16844,8281,BtyhcErvfI4z31FisBjeQg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.92660522460938 45.305442810058594),BtyhcErvfI4z31FisBjeQg,13312.0,6656.0,7561,3633,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605
16845,16595,--gG93FA53Lkv0H31-9Ruw,CurbRamp,NaN,True,True,False,True,POINT (-122.95636749267578 45.29937744140625),--gG93FA53Lkv0H31-9Ruw,13312.0,6656.0,1748,3983,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367


In [26]:
#select only the columns we need
labels = labels[['label_id','label_type','severity','gsv_panorama_id_x','zoom','heading','pitch','camera_heading','camera_pitch','user_id','lat','lng', 'geometry']]
#renmae gsv_panorama_id_x to gsv_panorama_id
labels = labels.rename(columns={'gsv_panorama_id_x': 'gsv_panorama_id'})

In [27]:
labels

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry
0,145,NoSidewalk,NaN,0R2JTLibSCNsNb7aGNUzWg,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904,POINT (-122.9529037475586 45.317649841308594)
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906)
2,151,NoSidewalk,NaN,oqg9ULHZul23xfU6DwTRNg,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598,POINT (-122.95359802246094 45.3176155090332)
3,153,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025,POINT (-122.95402526855469 45.317420959472656)
4,154,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185,POINT (-122.95418548583984 45.31760025024414)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,NoSidewalk,NaN,5KKnsHQgODUBbIRZYiVbqg,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797,POINT (-122.92579650878906 45.30332946777344)
16843,8272,NoSidewalk,NaN,mdbgEoi-XTfZGiKqcWMlqw,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591,POINT (-122.92559051513672 45.29985809326172)
16844,8281,NoSidewalk,NaN,BtyhcErvfI4z31FisBjeQg,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605,POINT (-122.92660522460938 45.305442810058594)
16845,16595,CurbRamp,NaN,--gG93FA53Lkv0H31-9Ruw,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367,POINT (-122.95636749267578 45.29937744140625)


In [28]:
#read in all files in the feature folder
clustered = pd.read_csv('../../../data/' + city + '/features/clustered.csv')
distance_to_road = pd.read_csv('../../../data/' + city + '/features/distance_to_road.csv')
distance_to_intersection = pd.read_csv('../../../data/' + city + '/features/distance_to_intersection.csv')
optional_input = pd.read_csv('../../../data/' + city + '/features/optional_input.csv')

In [29]:
#merge all into one on label_id
df = labels.merge(clustered, on='label_id').merge(distance_to_road, on='label_id').merge(distance_to_intersection, on='label_id').merge(optional_input, on='label_id')

In [30]:
#keep only the label types Curb Ramp, Missing Curb Ramp, Obstacle in Path, Surface Problem, and No Sidewalk
df = df[df['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem', 'NoSidewalk'])]

In [31]:
#drop nan in severity
df = df.dropna(subset=['severity'])

In [32]:
df

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906),10678,1,2,tertiary,36.874003,0.000000,1.0,3.0,0.0
7,18142,NoSidewalk,4.0,Xj2cLGwyFdmvW9764nWXLA,1,315.875000,-34.062500,86.560974,-0.588463,bb78d1ab-85d6-4620-a1de-396a5785080a,45.311462,-122.964020,POINT (-122.96401977539062 45.31146240234375),12062,1,2,residential,18.460328,0.000000,1.0,2.0,0.0
16,146,CurbRamp,1.0,0R2JTLibSCNsNb7aGNUzWg,1,50.125000,-26.937500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317612,-122.952751,POINT (-122.95275115966797 45.31761169433594),4305,1,3,tertiary,16.510172,32.896166,0.0,0.0,0.0
18,250,CurbRamp,2.0,SIRiUbNpH54imxj-z01Wnw,2,214.750000,-18.919643,270.914520,1.339951,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317303,-122.973167,POINT (-122.9731674194336 45.31730270385742),2275,1,3,secondary,32.557196,63.234151,1.0,1.0,0.0
19,22,CurbRamp,1.0,dslJGqaol13AzhdnPA1epA,1,157.750000,-34.437500,116.283493,-0.733246,a7b29eaa-8032-45f9-be2b-fce49aa0b8c0,45.320690,-122.972359,POINT (-122.97235870361328 45.3206901550293),2235,1,3,residential,17.679072,25.919038,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16817,10608,NoCurbRamp,3.0,Us_OPa9BYRG0w1HDzELG1Q,1,325.500000,0.000000,5.186546,-0.526199,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301994,-122.976204,POINT (-122.97620391845703 45.30199432373047),4460,1,3,residential,16.887283,28.193084,0.0,0.0,0.0
16820,10601,CurbRamp,1.0,uFwrAN91WDXKyIyOABLXCA,1,357.000000,-29.375000,356.907318,-0.724976,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301224,-122.976036,POINT (-122.97603607177734 45.30122375488281),102,1,3,residential,23.009559,41.952195,0.0,0.0,0.0
16821,10603,CurbRamp,1.0,uFwrAN91WDXKyIyOABLXCA,1,216.187500,-13.437500,356.907318,-0.724976,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301075,-122.976227,POINT (-122.97622680664062 45.30107498168945),94,1,3,primary,19.427711,32.225635,0.0,0.0,0.0
16822,11663,NoSidewalk,3.0,VTYJQEeA_nAjJcAuSjCIvw,1,186.857147,-23.375000,179.493881,1.367516,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.303471,-122.983231,POINT (-122.98323059082031 45.303470611572266),6571,0,1,residential,19.652903,0.000000,1.0,1.0,0.0


In [33]:
gt = pd.read_csv('../../../data/' + city + '/features/' + city + '_ground_truth_labels.csv')

In [34]:
gt

,label_id,ground_truth
0,8,1
1,9,1
2,10,1
3,11,1
4,12,1
...,...,...
2790,17672,0
2791,17674,1
2792,17676,1
2793,17688,1


In [35]:
#merge gt with labels how left
test= pd.merge(df, gt, on='label_id', how='inner')

In [36]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [37]:
test

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description,verified
0,22,CurbRamp,1.0,dslJGqaol13AzhdnPA1epA,1,157.7500,-34.4375,116.283493,-0.733246,a7b29eaa-8032-45f9-be2b-fce49aa0b8c0,45.320690,-122.972359,POINT (-122.97235870361328 45.3206901550293),2235,1,3,residential,17.679072,25.919038,0.0,0.0,0.0,1
1,23,CurbRamp,1.0,kI15oZgTQY8bxvRp-Io7nw,1,152.8750,-34.6250,44.911987,0.174957,a7b29eaa-8032-45f9-be2b-fce49aa0b8c0,45.320698,-122.972198,POINT (-122.97219848632812 45.32069778442383),2281,1,3,residential,14.490867,26.518458,0.0,0.0,0.0,1
2,25,NoCurbRamp,4.0,1IV2IjxbPQem-FE5LwIHFg,1,207.1250,-27.2500,221.050308,1.068039,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.297989,-122.979263,POINT (-122.97926330566406 45.29798889160156),5868,1,3,tertiary,14.894478,33.135027,1.0,1.0,0.0,1
3,26,NoCurbRamp,5.0,1IV2IjxbPQem-FE5LwIHFg,1,336.5000,-35.0000,221.050308,1.068039,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.298111,-122.979309,POINT (-122.97930908203125 45.29811096191406),6438,1,3,residential,14.826055,30.937320,1.0,1.0,0.0,1
4,29,NoCurbRamp,5.0,q5KyS5ERjAw3HJOHVigtBQ,1,227.9375,-29.5625,263.936096,0.018257,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.298023,-122.980125,POINT (-122.9801254272461 45.29802322387695),5984,0,1,residential,19.129094,30.510240,1.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714,10600,CurbRamp,1.0,pV6ygdJjUolRf7F9prg6hA,1,198.8125,-29.3750,91.113060,-0.234367,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.300262,-122.981255,POINT (-122.98125457763672 45.300262451171875),135,1,3,residential,17.092775,33.985072,0.0,0.0,0.0,1
2715,10602,CurbRamp,1.0,uFwrAN91WDXKyIyOABLXCA,1,302.4375,-23.9375,356.907318,-0.724976,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301216,-122.976219,POINT (-122.9762191772461 45.30121612548828),133,1,2,residential,24.076750,39.951514,0.0,0.0,0.0,1
2716,10606,SurfaceProblem,3.0,q7ZWOSCwmELNFWLcdOMbSQ,1,346.1250,0.0000,0.209879,-1.773552,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301556,-122.976227,POINT (-122.97622680664062 45.30155563354492),14806,1,2,residential,27.132079,0.000000,1.0,2.0,0.0,1
2717,10607,NoCurbRamp,3.0,Us_OPa9BYRG0w1HDzELG1Q,1,325.5000,0.0000,5.186546,-0.526199,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301907,-122.976196,POINT (-122.9761962890625 45.30190658569336),4464,1,3,residential,15.090810,25.419408,0.0,0.0,0.0,1


In [38]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [39]:
training

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906),10678,1,2,tertiary,36.874003,0.000000,1.0,3.0,0.0
7,18142,NoSidewalk,4.0,Xj2cLGwyFdmvW9764nWXLA,1,315.875000,-34.062500,86.560974,-0.588463,bb78d1ab-85d6-4620-a1de-396a5785080a,45.311462,-122.964020,POINT (-122.96401977539062 45.31146240234375),12062,1,2,residential,18.460328,0.000000,1.0,2.0,0.0
16,146,CurbRamp,1.0,0R2JTLibSCNsNb7aGNUzWg,1,50.125000,-26.937500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317612,-122.952751,POINT (-122.95275115966797 45.31761169433594),4305,1,3,tertiary,16.510172,32.896166,0.0,0.0,0.0
18,250,CurbRamp,2.0,SIRiUbNpH54imxj-z01Wnw,2,214.750000,-18.919643,270.914520,1.339951,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317303,-122.973167,POINT (-122.9731674194336 45.31730270385742),2275,1,3,secondary,32.557196,63.234151,1.0,1.0,0.0
27,147,CurbRamp,1.0,0R2JTLibSCNsNb7aGNUzWg,2,72.830360,-13.714286,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317596,-122.952675,POINT (-122.95267486572266 45.317596435546875),1303,1,4,secondary,24.667766,40.796348,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16816,10599,CurbRamp,1.0,pV6ygdJjUolRf7F9prg6hA,1,198.812500,-29.375000,91.113060,-0.234367,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.300274,-122.981277,POINT (-122.98127746582031 45.30027389526367),112,1,3,residential,23.027326,34.185393,0.0,0.0,0.0
16820,10601,CurbRamp,1.0,uFwrAN91WDXKyIyOABLXCA,1,357.000000,-29.375000,356.907318,-0.724976,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301224,-122.976036,POINT (-122.97603607177734 45.30122375488281),102,1,3,residential,23.009559,41.952195,0.0,0.0,0.0
16821,10603,CurbRamp,1.0,uFwrAN91WDXKyIyOABLXCA,1,216.187500,-13.437500,356.907318,-0.724976,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301075,-122.976227,POINT (-122.97622680664062 45.30107498168945),94,1,3,primary,19.427711,32.225635,0.0,0.0,0.0
16822,11663,NoSidewalk,3.0,VTYJQEeA_nAjJcAuSjCIvw,1,186.857147,-23.375000,179.493881,1.367516,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.303471,-122.983231,POINT (-122.98323059082031 45.303470611572266),6571,0,1,residential,19.652903,0.000000,1.0,1.0,0.0


In [40]:
training.to_csv('../../../data/' + city + '/processed-labels/training_set_' + city + '.csv', index=False)
test.to_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv', index=False)
df.to_csv('../../../data/' + city + '/processed-labels/labels_full_features_' + city + '.csv', index=False)